In [1]:
import pandas as pd
import numpy as np


In [2]:

from sklearn.model_selection import train_test_split

# Train and Test sets

## Training set (imputing and normalizing)

In [3]:
df = pd.read_excel("3.xlsx")

In [66]:
# ooutcome columns: 1.are Outcome_InhospitalMortality 2.TM_S_Intubation  3.Outcome_ICUadmission  4.TM_S_Dialysis
df = df.dropna(subset=['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis'])

In [67]:
#inja oomadim va external validation ro freeze kardim
df_internal_validation = df[df['Patient_Hospital']!="Hospital2"]
df_external_validation = df[df['Patient_Hospital']=="Hospital2"]


df_internal_validation = df_internal_validation.drop(columns=['Patient_Hospital'])
df_external_validation = df_external_validation.drop(columns=['Patient_Hospital'])

In [68]:
#df.to_excel("excluding NAN.xlsx", index=False)

In [69]:
#inja train test split kardim va target variable ro jodakardim
# Define features (X) and target variable (y)
X = df_internal_validation.drop(columns=['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis'])
y  = df_internal_validation[['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis']]
# Perform the train-test split (adjust the test_size as needed)
X_train , X_test , y_train , y_test = train_test_split(X,y , test_size = 0.2 , random_state = 42)

In [70]:
#inja miaim va data haye categorical va numerical ro moshakhas mikonim ke bade knn categorical ha ro round konim
cat_col=[]
num_col=[]
for i in range(len(X_train.columns)):
    if i <=52:
        cat_col.append(X_train.columns[i])
    else:
        num_col.append(X_train.columns[i])


In [71]:
from sklearn.impute import KNNImputer
X_train_array  =X_train.to_numpy()
imputer = KNNImputer(n_neighbors=5, weights="uniform")
imputed = imputer.fit_transform(X_train_array)

In [72]:
df_imputed = pd.DataFrame(imputed)
df_imputed.columns=pd.DataFrame(X_train).columns.tolist()


In [73]:
for i in cat_col:
    df_imputed[[i]] = df_imputed[[i]].round()

In [74]:
df_imputed.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

In [75]:
#miaim x , y ro be ham michaboonim ta betonim normilize konim
df_before_normalize = pd.concat([df_imputed , y_train], axis=1)

In [76]:
arr_tain =df_imputed.to_numpy()

In [77]:
from sklearn import preprocessing
transform = preprocessing.StandardScaler()
df_normalize = transform.fit_transform(arr_tain)
df_normalize =pd.DataFrame(df_normalize)
df_normalize.columns = df_imputed.columns.tolist()

In [78]:
X_train = df_normalize
y_train = df_before_normalize[['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis']]

## Test set (imputing and normalizing)

In [79]:
#inja miaim va data haye categorical va numerical ro moshakhas mikonim ke bade knn categorical ha ro round konim
cat_col=[]
num_col=[]
for i in range(len(X_test.columns)):
    if i <=52:
        cat_col.append(X_test.columns[i])
    else:
        num_col.append(X_test.columns[i])


In [80]:
from sklearn.impute import KNNImputer
X_test_array  =X_test.to_numpy()
imputer = KNNImputer(n_neighbors=5, weights="uniform")
imputed_test = imputer.fit_transform(X_test_array)

In [81]:
df_imputed_test = pd.DataFrame(imputed_test)
df_imputed_test.columns=pd.DataFrame(X_test).columns.tolist()


In [82]:
for i in cat_col:
    df_imputed_test[[i]] = df_imputed_test[[i]].round()

In [83]:
df_imputed_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [84]:
#miaim x , y ro be ham michaboonim ta betonim normilize konim
df_test_before_normalize = pd.concat([df_imputed_test , y_test], axis=1)

In [85]:
arr =df_test_before_normalize.to_numpy()

In [86]:
from sklearn import preprocessing
transform = preprocessing.StandardScaler()
df_test_normalize = transform.fit_transform(arr)
df_test_normalize =pd.DataFrame(df_test_normalize)
df_test_normalize.columns = df_internal_validation.columns.tolist()


In [87]:
X_test = df_test_normalize.drop(columns=['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis'])
y_test = df_test_before_normalize[['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis']]

>>>Now dataset is cleaned and normalized 

# experimenting different models on dataset

In [88]:

import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [89]:
X_train = np.array(X_train)
X_test = np.array(X_test)

## Logistic Regression

In [90]:

#remember to consider x_test and y_test for each model

In [91]:
y_train_Outcome_InhospitalMortality = np.array(y_train['Outcome_InhospitalMortality'].tolist())
y_test_Outcome_InhospitalMortality = np.array(y_test['Outcome_InhospitalMortality'].tolist())

In [92]:
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)
def logistic_regression_classifier(X_train, y_train, X_test, y_test):

    parameters_lr = {}


    lr = LogisticRegression(random_state=42)


    grid_search_lr = GridSearchCV(
        estimator=lr,
        param_grid=parameters_lr,
        cv=5,
        n_jobs=-1
    )

    logreg_cv = grid_search_lr.fit(X_train, y_train)

    
    best_classifier_lr = logreg_cv.best_estimator_

 
    y_predicted_Mortality_lr = best_classifier_lr.predict(X_test)

 
    accuracy = accuracy_score(y_test, y_predicted_Mortality_lr)
    cm = confusion_matrix(y_test, y_predicted_Mortality_lr)
    precision = precision_score(y_test, y_predicted_Mortality_lr)
    recall = recall_score(y_test, y_predicted_Mortality_lr)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_lr)


    Logistic_Regression = {}
    Logistic_Regression["accuracy"] = accuracy
    Logistic_Regression["precision"] = precision
    Logistic_Regression["recall"] = recall
    Logistic_Regression["specificity"] = specificity
    Logistic_Regression["f1"] = f1

    y_plot = {}
    y_plot["y_true"] = y_train
    y_plot["y_predicted"] = y_predicted_Mortality_lr
    
    Logistic_Regression = (Logistic_Regression, y_plot)
    
    return Logistic_Regression


## SVM

In [93]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

def train_and_evaluate_svm(X_train, y_train, X_test, y_test):

    svm_classifier = svm.SVC(random_state=1234)


    parameters_svm = {}


    grid_search_svm = GridSearchCV(
        estimator=svm_classifier,
        param_grid=parameters_svm,
        cv=5
    )
    
   
    svm_cv = grid_search_svm.fit(X_train, y_train)

 
    best_classifier_svm = svm_cv.best_estimator_


    y_predicted_Mortality_svm = best_classifier_svm.predict(X_test)

   
    accuracy = accuracy_score(y_test, y_predicted_Mortality_svm)
    cm = confusion_matrix(y_test, y_predicted_Mortality_svm)
    precision = precision_score(y_test, y_predicted_Mortality_svm)
    recall = recall_score(y_test, y_predicted_Mortality_svm)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_svm)

    svm_results = {}
    svm_results["accuracy"] = accuracy
    svm_results["precision"] = precision
    svm_results["recall"] = recall
    svm_results["specificity"] = specificity
    svm_results["f1"] = f1

  
    y_plot = {}
    y_plot["y_true"] = y_train  
    y_plot["y_predicted"] = y_predicted_Mortality_svm

    svm_results = (svm_results, y_plot)
    
    return svm_results



## Decision tree

In [94]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

def train_and_evaluate_tree(X_train, y_train, X_test, y_test):

    parameters_tree = {}

 
    tree = DecisionTreeClassifier(random_state=12345)

 
    grid_search_tree = GridSearchCV(
        estimator=tree,
        param_grid=parameters_tree,
        cv=5
    )
    

    tree_cv = grid_search_tree.fit(X_train, y_train)

  
    best_classifier_tree = tree_cv.best_estimator_

 
    y_predicted_Mortality_tree = best_classifier_tree.predict(X_test)

 
    accuracy = accuracy_score(y_test, y_predicted_Mortality_tree)
    cm = confusion_matrix(y_test, y_predicted_Mortality_tree)
    precision = precision_score(y_test, y_predicted_Mortality_tree)
    recall = recall_score(y_test, y_predicted_Mortality_tree)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_tree)

  
    tree_results = {}
    tree_results["accuracy"] = accuracy
    tree_results["precision"] = precision
    tree_results["recall"] = recall
    tree_results["specificity"] = specificity
    tree_results["f1"] = f1


    y_plot = {}
    y_plot["y_true"] = y_train 
    y_plot["y_predicted"] = y_predicted_Mortality_tree

    tree_results = (tree_results, y_plot)
    
    return tree_results



## KNN

In [95]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

def train_and_evaluate_knn(X_train, y_train, X_test, y_test):

    parameters_knn = {}

 
    knn = KNeighborsClassifier()


    grid_search_knn = GridSearchCV(
        estimator=knn,
        param_grid=parameters_knn,
        cv=5
    )
    

    knn_cv = grid_search_knn.fit(X_train, y_train)

    best_classifier_knn = knn_cv.best_estimator_

    y_predicted_Mortality_knn = best_classifier_knn.predict(X_test)

    accuracy = accuracy_score(y_test, y_predicted_Mortality_knn)
    cm = confusion_matrix(y_test, y_predicted_Mortality_knn)
    precision = precision_score(y_test, y_predicted_Mortality_knn)
    recall = recall_score(y_test, y_predicted_Mortality_knn)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_knn)

    knn_results = {}
    knn_results["accuracy"] = accuracy
    knn_results["precision"] = precision
    knn_results["recall"] = recall
    knn_results["specificity"] = specificity
    knn_results["f1"] = f1


    y_plot = {}
    y_plot["y_true"] = y_train  
    y_plot["y_predicted"] = y_predicted_Mortality_knn

    knn_results = (knn_results, y_plot)
    
    return knn_results



## Random forest

In [96]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

def train_and_evaluate_forest(X_train, y_train, X_test, y_test):
    parameters_forest = {}


    forest = RandomForestClassifier()


    grid_search_forest = GridSearchCV(
        estimator=forest,
        param_grid=parameters_forest,
        cv=5
    )
    

    forest_cv = grid_search_forest.fit(X_train, y_train)

    best_classifier_forest = forest_cv.best_estimator_

    y_predicted_Mortality_forest = best_classifier_forest.predict(X_test)


    accuracy = accuracy_score(y_test, y_predicted_Mortality_forest)
    cm = confusion_matrix(y_test, y_predicted_Mortality_forest)
    precision = precision_score(y_test, y_predicted_Mortality_forest)
    recall = recall_score(y_test, y_predicted_Mortality_forest)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_forest)


    forest_results = {}
    forest_results["accuracy"] = accuracy
    forest_results["precision"] = precision
    forest_results["recall"] = recall
    forest_results["specificity"] = specificity
    forest_results["f1"] = f1

    y_plot = {}
    y_plot["y_true"] = y_train
    y_plot["y_predicted"] = y_predicted_Mortality_forest

    return forest_results, y_plot


## Neural Network

In [97]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

def train_and_evaluate_neural(X_train, y_train, X_test, y_test):

    parameters_neural = {}


    neural = MLPClassifier()

    grid_search_neural = GridSearchCV(
        estimator=neural,
        param_grid=parameters_neural,
        cv=5
    )
    

    neural_cv = grid_search_neural.fit(X_train, y_train)

   
    best_classifier_neural = neural_cv.best_estimator_

    
    y_predicted_Mortality_neural = best_classifier_neural.predict(X_test)

    accuracy = accuracy_score(y_test, y_predicted_Mortality_neural)
    cm = confusion_matrix(y_test, y_predicted_Mortality_neural)
    precision = precision_score(y_test, y_predicted_Mortality_neural)
    recall = recall_score(y_test, y_predicted_Mortality_neural)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_neural)

 
    neural_results = {}
    neural_results["accuracy"] = accuracy
    neural_results["precision"] = precision
    neural_results["recall"] = recall
    neural_results["specificity"] = specificity
    neural_results["f1"] = f1

    
    y_plot = {}
    y_plot["y_true"] = y_train  
    y_plot["y_predicted"] = y_predicted_Mortality_neural

    return neural_results, y_plot



## XGboost

In [98]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

def train_and_evaluate_boost(X_train, y_train, X_test, y_test):
  
    parameters_boost = {}

 
    boost = XGBClassifier()


    grid_search_boost = GridSearchCV(
        estimator=boost,
        param_grid=parameters_boost,
        cv=5
    )
    

    boost_cv = grid_search_boost.fit(X_train, y_train)


    best_classifier_boost = boost_cv.best_estimator_


    y_predicted_Mortality_boost = best_classifier_boost.predict(X_test)


    accuracy = accuracy_score(y_test, y_predicted_Mortality_boost)
    cm = confusion_matrix(y_test, y_predicted_Mortality_boost)
    precision = precision_score(y_test, y_predicted_Mortality_boost)
    recall = recall_score(y_test, y_predicted_Mortality_boost)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_boost)

  
    boost_results = {}
    boost_results["accuracy"] = accuracy
    boost_results["precision"] = precision
    boost_results["recall"] = recall
    boost_results["specificity"] = specificity
    boost_results["f1"] = f1


    y_plot = {}
    y_plot["y_true"] = y_train 
    y_plot["y_predicted"] = y_predicted_Mortality_boost

    return boost_results, y_plot



# Final results

In [99]:
list_Outcome_InhospitalMortality=[]

y_train = df_before_normalize['Outcome_InhospitalMortality']
y_test = df_test_before_normalize['Outcome_InhospitalMortality']

In [100]:

X_train = df_normalize
X_test = df_test_normalize.drop(columns=['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis'])

y_train_ = pd.DataFrame(y_train)
y_test_ = pd.DataFrame(y_test)



In [101]:
X_train.shape

(6118, 76)

In [102]:
X_test.shape

(1530, 76)

In [103]:
y_train_.shape

(6118, 1)

In [105]:
y_test_

,Outcome_InhospitalMortality
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
1525,1.0
1526,0.0
1527,1.0
1528,1.0


## data selection Mortality Undersampling and and Oversampling

### feature selection 

In [60]:
#Lasso 


from sklearn.linear_model import Lasso

def lasso(X_train ,y_train,X_test):
    # Perform feature selection using LASSO on the training set
    lasso = Lasso(alpha=0.1)
    lasso.fit(X_train, y_train)

    # Get the absolute coefficients and sort them
    absolute_coeffs = np.abs(lasso.coef_)
    sorted_indices = np.argsort(absolute_coeffs)[::-1]
    
    
    X_train =pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    # Select the top 2 features with the highest coefficients
    selected_features = X_train.columns[sorted_indices[:40]]

    # Apply feature selection to both the training and test sets
    X_train_selected = X_train[selected_features]
    X_test_selected= X_test[selected_features]
    
    return (pd.DataFrame(X_train_selected), pd.DataFrame(X_test_selected))


In [65]:

lasso(pd.DataFrame(X_train) ,pd.DataFrame(y_train),pd.DataFrame(X_test))

ValueError: Multi-dimensional indexing (e.g. `obj[:, None]`) is no longer supported. Convert to a numpy array before indexing instead.

In [44]:
#RFE

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

def rfe(X_train, y_train, X_test):
    # Create a logistic regression model
    model = LogisticRegression()
    
    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)

    # Perform RFE to select top 'n_features' features
    rfe = RFE(estimator=model, n_features_to_select=40)
    X_train_selected = rfe.fit_transform(X_train, y_train)
    X_test_selected = rfe.transform(X_test)

    return (pd.DataFrame(X_train_selected), pd.DataFrame(X_test_selected))



In [45]:
# RandomForestClassifier

from sklearn.ensemble import RandomForestClassifier

def random_forest_feature_importance(X_train, y_train, X_test):
    # Create a random forest classifier
    model = RandomForestClassifier()

    # Fit the model to obtain feature importances
    model.fit(X_train, y_train)

    # Get feature importances and sort them in descending order
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    X_train  = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    # Select top 'n_features' features based on importance
    selected_features = X_train.columns[indices][:40]

    # Apply feature selection to both the training and test sets
    X_train_selected = X_train[selected_features]
    X_test_selected = X_test[selected_features]

    return (pd.DataFrame(X_train_selected), pd.DataFrame(X_test_selected))

In [46]:
list_fun_feature = [lasso, rfe, random_forest_feature_importance]


### handeling imbalence data

In [47]:
#SMOTETomek 

from imblearn.combine import SMOTETomek
def SMOTETomek_mix(X_train,y_train , X_test , y_test):
    
    tl = SMOTETomek(random_state=42)
    X_resampled_train , y_resampled_train = tl.fit_resample(X_train,y_train)
    X_resampled_test , y_resampled_test = tl.fit_resample(X_test,y_test)
    
    return (X_resampled_train, y_resampled_train,X_resampled_test , y_resampled_test )

In [48]:
#RandomUnderSampler
from imblearn.under_sampling import RandomUnderSampler

def random_undersampling(X_train,y_train , X_test , y_test):
    # Create a random undersampler
    rus = RandomUnderSampler(random_state=42)

    # Perform random undersampling
    X_resampled_train , y_resampled_train = rus.fit_resample(X_train,y_train)
    X_resampled_test , y_resampled_test = rus.fit_resample(X_test,y_test)
    
    return (X_resampled_train, y_resampled_train,X_resampled_test , y_resampled_test )

In [49]:
#RandomOverSampler

from imblearn.over_sampling import RandomOverSampler

def random_oversampling(X_train,y_train , X_test , y_test):
    # Create a random oversampler
    ros = RandomOverSampler(random_state=42)

    # Perform random oversampling
    X_resampled_train , y_resampled_train = ros.fit_resample(X_train,y_train)
    X_resampled_test , y_resampled_test = ros.fit_resample(X_test,y_test)
    return (X_resampled_train, y_resampled_train,X_resampled_test , y_resampled_test )

In [50]:
list_fun_imbalance = [SMOTETomek_mix, RandomUnderSampler, random_oversampling]


In [51]:
list_fun_feature = [lasso, rfe, random_forest_feature_importance]
list_fun_imbalance = [SMOTETomek_mix, RandomUnderSampler, random_oversampling]

feature_dict_keys = ['SMOTETomek_mix', 'RandomUnderSampler', 'random_oversampling']

l = 0
feature_dict={}
for fun_feature in list_fun_feature:
    
    X_train = df_normalize
    y_train = df_before_normalize[['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis']]
    X_test = df_test_normalize.drop(columns=['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis'])
    y_test = df_test_before_normalize[['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis']]
    y_train_ = pd.DataFrame(y_train['Outcome_InhospitalMortality'])
    y_test_ = pd.DataFrame(y_test['Outcome_InhospitalMortality'])
    
    imbalance_list=[]
    for fun_imbalanced  in list_fun_imbalance:
        
        X_train=np.array(X_train)
        X_test=np.array(X_test)

        y_train_ = np.array(y_train)
        y_test_ = np.array(y_test)
        
        X_train, X_test =  fun_feature(X_train, y_train, X_test)
        X_train,y_train_ , X_test , y_test_ = fun_imbalanced(X_train,y_train_ , X_test , y_test_)
        
        imbalance_list.append((X_train,y_train_ , X_test , y_test_))
        
    print(imbalance_list)


ValueError: Multi-dimensional indexing (e.g. `obj[:, None]`) is no longer supported. Convert to a numpy array before indexing instead.

In [ ]:
lis = ["n" ,"m"]

for a,b in lis,range(len(lis)):
    print(a,b)
    print(b)

n m
m
0 1
1


In [ ]:
list_Outcome_InhospitalMortality=[]
X_train, X_test = lasso_MO_ICU(pd.DataFrame(X_train), y_train_ ,pd.DataFrame(X_test))

X_train=np.array(X_train)
X_test=np.array(X_test)

y_train_ = np.array(y_train['Outcome_InhospitalMortality'])
y_test_ = np.array(y_test['Outcome_InhospitalMortality'])

X_train,y_train_ , X_test , y_test_ = balancing_dataset(X_train,y_train_ , X_test , y_test_)
    
list_Outcome_InhospitalMortality.extend([logistic_regression_classifier(X_train, y_train_, X_test, y_test_),
                                            train_and_evaluate_svm(X_train, y_train_, X_test, y_test_),
                                        train_and_evaluate_tree(X_train, y_train_, X_test, y_test_),
                                        train_and_evaluate_knn(X_train, y_train_, X_test, y_test_),
                                        train_and_evaluate_forest(X_train, y_train_, X_test, y_test_),
                                        train_and_evaluate_neural(X_train, y_train_, X_test, y_test_),
                                        train_and_evaluate_boost(X_train, y_train_, X_test, y_test_)])



In [ ]:
result_dic_list_Outcome_InhospitalMortality = dict(zip(['logistic_regression', 'svm', 'tree', 'knn', 'forest', 'neural', 'boost'], list_Outcome_InhospitalMortality))


In [ ]:

merged_dict = {}

# List of dictionary names and their corresponding dictionaries
dict_list = [('Outcome_InhospitalMortality', result_dic_list_Outcome_InhospitalMortality)]

# Merge the dictionaries
for name, result_dict in dict_list:
    merged_dict[name] = result_dict

# The merged_dict now contains all the dictionaries merged together

In [ ]:
merged_dict

In [ ]:
import pandas as pd


df = pd.DataFrame(merged_dict)


df = df.transpose()


df.reset_index(inplace=True)
df.rename(columns={'index': 'Method'}, inplace=True)



In [ ]:
df

In [ ]:
data = []

for outcome, models in merged_dict.items():
    for model, metrics in models.items():
        accuracy, precision, recall, specificity, f1 = metrics[0]['accuracy'], metrics[0]['precision'], metrics[0]['recall'], metrics[0]['specificity'], metrics[0]['f1']
        y_true, y_predicted = metrics[1]['y_true'], metrics[1]['y_predicted']
        data.append([outcome, model, accuracy, precision, recall, specificity, f1, y_true.tolist(), y_predicted.tolist()])

columns = ['Outcome', 'Model', 'Accuracy', 'Precision', 'Recall', 'Specificity', 'F1', 'y_true', 'y_predicted']

df = pd.DataFrame(data, columns=columns)


In [ ]:
df